# Générateur de fiche de paye Pajemploi

## Setup

In [ ]:
from datetime import datetime
import utils, paidleave

## Params

In [ ]:
year = 2023
month = "juin"

## Charger les données

Charge la configuration (détails du contrat de l'auxiliaire) présente au path `data/{year}/{month}/config.json` et le calendrier des absences au path `data/{year}/{month}/calendars.json`

Un example de format est disponible.

**schéma de données pour la config :**
- `start_date_period` : date de départ de la période pour laquelle ce contrat s'applique.
- `end_date_period` : date de fin de la période pour laquelle ce contrat s'applique.
- `hourly_salary` : salaire horaire **NET**.
- `worked_weekly_hours` : nombre total d'heures hebdomadaires travaillées (max 50 légalement).
- `recovered_weekly_hours` : nombre d'heures hebdomadaires "récupérées" en jours non-travaillés.
- `transport_monthly_cost` : frais de transport mensuels de l'auxiliaire parentale.
- `transport_allowance_prct` : part des frais de transport remboursés par les employeurs (usuellement 100%).
- `daily_meal_allowance` : indemnité de repas quotidienne (par jour travaillé).
- `allocation_key` : clé de répartition des frais sur les différentes famille en cas de garde partagée.

Le schéma de données permet de donner plusieurs configs dans le cas ou un changement de contrat a eu lieu en cours de mois. Dans ce cas, faîtes bien attention à ce que les périodes temporelles soient bien disjointes.

**schéma de données pour les absences :**

- `vacation_calendar` : liste des périodes de congés payés et/ou jours de récupération sur le mois.
    - `start_date_included` : premier jour de vacance de la période
    - `end_date_excluded` : premier jour de **retour au travail** de l'auxiliaire parentale
    - **Attention** : Pour signifier que l'auxiliaire parentale a été absente une demi-journée seulement, inscriver la même date pour `start_date_included` et `end_date_excluded`.
- `holidays` : liste des jours fériés sur la période.
- `sick_days` : liste des périodes de congés maladie sur le mois.
    - `start_date_included` : premier jour de vacance de la période
    - `end_date_excluded` : premier jour de **retour au travail** de l'auxiliaire parentale
    - **Attention** : Pour signifier que l'auxiliaire parentale a été absente une demi-journée seulement, inscriver la même date pour `start_date_included` et `end_date_excluded`.

Si vous n'avez pas d'absence à signaler ce mois-ci dans l'une ou l'autre catégorie, laisser la liste vide : `[]`

In [ ]:
configs = utils.load_config(year, month)

In [ ]:
vacation_calendar, holidays, sick_days = utils.load_calendars(year, month)

## Revue du mois

In [ ]:
start_date = datetime.strptime(configs[0]['start_date_period'], "%Y-%m-%d").date()
end_date = datetime.strptime(configs[len(configs)-1]['end_date_period'], "%Y-%m-%d").date()

In [ ]:
df = utils.display_monthly_review(
    start_date=start_date,
    end_date=end_date, 
    vacation_calendar=vacation_calendar, 
    holidays=holidays, 
    sick_days=sick_days
)

## Déclaration Pajemploi

In [ ]:
period_wages, cp_aquired, recovered_days_acquired = utils.compute_period_wages(start_date, end_date, configs, vacation_calendar=vacation_calendar, holidays=holidays, sick_days=sick_days)

In [ ]:
total_wages = utils.compute_total_wages(period_wages)
total_wages

In [ ]:
utils.display_urssaf_input_values(total_wages, 'Martin')

In [ ]:
utils.display_paid_leave_acquired(cp_aquired, recovered_days_acquired)

In [ ]:
paidleave.update_suivi(
    year=year,
    month=month,
    cp_aquired=cp_aquired, 
    recovered_days_acquired=recovered_days_acquired,
    days_off_used=df['working_vacation'].sum()
)

In [ ]:
paidleave.display_paid_leave_accumulated()